## Demo

In [1]:
import main
import json

## базовий приклад з 4 тайм слотами

In [3]:
# Example with microservices (6 time slots)
print("\n" + "="*60)
print("Input data:")

microservices = [
    [1, 2, 3, 4],  # Microservice 0
    [2, 1, 1, 2],  # Microservice 1
    [3, 3, 2, 1],  # Microservice 2
]

for i, service in enumerate(microservices):
    print(f"Microservice {i}: {service}")
print("="*60 + "\n")

try:
    groups, group_services, slot_sums = main.form_multiple_knapsack_groups(microservices)
    main.print_results(groups, group_services, slot_sums)
except ValueError as e:
    print(f"Error: {e}")


Input data:
Microservice 0: [1, 2, 3, 4]
Microservice 1: [2, 1, 1, 2]
Microservice 2: [3, 3, 2, 1]

Grouping results:

Group 1:
  Microservices: [1, 2, 0]
  Individual load by time slots:
    Microservice 1: [2, 1, 1, 2]
    Microservice 2: [3, 3, 2, 1]
    Microservice 0: [1, 2, 3, 4]
  Total load by time slots: [6, 6, 6, 7]
  Average load: 6.25
  Standard deviation: 0.43
  Coefficient of variation: 6.93%

Overall statistics:
  Number of groups: 1
  Total number of microservices: 3
  Average coefficient of variation: 6.93%


## базовий приклад з 6 тайм слотами

In [5]:
print("\n" + "="*60)
print("Input data:")
    
microservices = [
    [1, 2, 3, 4, 3, 2],  # Microservice 0
    [2, 1, 1, 2, 3, 4],  # Microservice 1
    [3, 3, 2, 1, 2, 3],  # Microservice 2
    [4, 3, 2, 1, 1, 2],  # Microservice 3
    [2, 3, 4, 3, 2, 1],  # Microservice 4
    [1, 2, 3, 4, 4, 3],  # Microservice 5
    [3, 2, 1, 2, 3, 4],  # Microservice 6
    [4, 3, 2, 1, 2, 3],  # Microservice 7
]
    
for i, service in enumerate(microservices):
    print(f"Microservice {i}: {service}")

print("="*60 + "\n")

try:
    groups, group_services, slot_sums = main.form_multiple_knapsack_groups(microservices)
    main.print_results(groups, group_services, slot_sums)
except ValueError as e:
    print(f"Error: {e}")
        


Input data:
Microservice 0: [1, 2, 3, 4, 3, 2]
Microservice 1: [2, 1, 1, 2, 3, 4]
Microservice 2: [3, 3, 2, 1, 2, 3]
Microservice 3: [4, 3, 2, 1, 1, 2]
Microservice 4: [2, 3, 4, 3, 2, 1]
Microservice 5: [1, 2, 3, 4, 4, 3]
Microservice 6: [3, 2, 1, 2, 3, 4]
Microservice 7: [4, 3, 2, 1, 2, 3]

Grouping results:

Group 1:
  Microservices: [6, 4, 3, 5]
  Individual load by time slots:
    Microservice 6: [3, 2, 1, 2, 3, 4]
    Microservice 4: [2, 3, 4, 3, 2, 1]
    Microservice 3: [4, 3, 2, 1, 1, 2]
    Microservice 5: [1, 2, 3, 4, 4, 3]
  Total load by time slots: [10, 10, 10, 10, 10, 10]
  Average load: 10.00
  Standard deviation: 0.00
  Coefficient of variation: 0.00%

Group 2:
  Microservices: [0, 2, 1, 7]
  Individual load by time slots:
    Microservice 0: [1, 2, 3, 4, 3, 2]
    Microservice 2: [3, 3, 2, 1, 2, 3]
    Microservice 1: [2, 1, 1, 2, 3, 4]
    Microservice 7: [4, 3, 2, 1, 2, 3]
  Total load by time slots: [10, 9, 8, 8, 10, 12]
  Average load: 9.50
  Standard deviation: 1

## приклад з 24 тайм слотами

(тут генерував дані мікросервісів аі-шкою)

In [15]:
with open('testing/realistic_data.json', 'r') as f:
    real_data = json.load(f)
    
print(f"Завантажено {len(real_data)} мікросервісів з реальними патернами навантаження")

# Запуск алгоритму групування
groups, group_services, slot_sums = main.form_multiple_knapsack_groups(real_data)
print(f"Алгоритм створив {len(groups)} груп")
main.print_results(groups, group_services, slot_sums)

# Аналіз стабільності груп
group_stats = []
for i, (group, services, sums) in enumerate(zip(groups, group_services, slot_sums)):
    mean = sum(sums) / len(sums)
    variance = sum((x - mean) ** 2 for x in sums) / len(sums)
    std_dev = variance ** 0.5
    cv = (std_dev / mean) * 100 if mean > 0 else float('inf')
    
    group_stats.append({
        'group_id': i,
        'size': len(services),
        'mean': mean,
        'std_dev': std_dev,
        'cv': cv,
        'services': services
    })

# Сортування груп за стабільністю (CV)
group_stats.sort(key=lambda x: x['cv'])

# Вивід статистики
print("\nСтатистика груп (відсортована за коефіцієнтом варіації):")
print(f"{'ID групи':<10} {'Розмір':<10} {'Сер. навант.':<15} {'Станд. відх.':<15} {'Коеф. варіації':<15}")
print("-" * 65)

for stat in group_stats[:10]:  # Виводимо тільки 10 найкращих груп
    print(f"{stat['group_id']:<10} {stat['size']:<10} {stat['mean']:.2f}{' '*11} {stat['std_dev']:.2f}{' '*11} {stat['cv']:.2f}%")


Завантажено 50 мікросервісів з реальними патернами навантаження
Алгоритм створив 14 груп
Grouping results:

Group 1:
  Microservices: [4, 3, 41, 6, 5]
  Individual load by time slots:
    Microservice 4: [3.7, 3.6, 3.7, 3.6, 4.4, 4.3, 3.8, 4.0, 4.0, 3.8, 4.0, 3.6, 4.0, 3.7, 4.2, 3.9, 3.7, 3.8, 4.3, 3.8, 4.4, 4.1, 3.6, 3.7]
    Microservice 3: [3.1, 3.3, 3.8, 3.8, 3.2, 3.3, 3.8, 3.0, 3.4, 3.7, 3.8, 3.2, 3.9, 3.8, 3.6, 3.1, 3.6, 3.6, 3.2, 3.4, 3.2, 3.7, 3.7, 3.7]
    Microservice 41: [3.7, 4.2, 3.5, 3.6, 3.6, 3.7, 4.3, 4.2, 4.2, 3.6, 3.7, 3.3, 3.3, 4.1, 3.9, 3.7, 3.7, 4.1, 3.7, 3.8, 4.2, 3.4, 3.9, 3.8]
    Microservice 6: [3.9, 3.4, 4.1, 4.0, 4.0, 3.9, 3.3, 3.4, 3.4, 4.0, 3.9, 4.3, 4.3, 3.5, 3.7, 3.9, 3.9, 3.5, 3.9, 3.8, 3.4, 4.2, 3.7, 3.8]
    Microservice 5: [3.1, 2.6, 2.5, 3.0, 2.4, 2.7, 2.1, 2.3, 3.1, 2.7, 2.3, 2.7, 2.3, 2.3, 2.7, 2.7, 3.1, 2.5, 2.7, 2.6, 2.6, 3.0, 2.8, 3.1]
  Total load by time slots: [17.5, 17.1, 17.6, 18.0, 17.6, 17.900000000000002, 17.3, 16.9, 18.1, 17.8, 17.7, 1